In [32]:
import pandas as pd
import numpy as np
import os
import glob

In [34]:
################# lecture des donées ######################
csv_files = glob.glob("C:/Users/TRETEC/Desktop/DARI project/transactions-ser/*.csv")
data=pd.DataFrame()
L=[]
for f in csv_files:    
    L.append(pd.read_csv(f,sep=";"))
data=pd.concat(L)
###########################################################